In [ ]:

import numpy as np
import pandas as pd

from itertools import combinations
from scipy.stats import kruskal

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

random_state = 67


## 1. Data loading

In [6]:

# Load data (adjust path if needed)
data_path = "dataR2.csv"
DataSet = pd.read_csv(data_path)

# Ensure expected target name is present
assert "Classification" in DataSet.columns, "Expected 'Classification' column not found."

# Separate X / y
X = DataSet.drop(columns=["Classification"]).copy()
y = DataSet["Classification"].copy()

print("Shape:", X.shape)
display(DataSet.head())
print("\nClass balance:")
print(y.value_counts().rename({1:"Healthy", 2:"With Cancer"}))


Shape: (116, 9)


,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1



Class balance:
Classification
With Cancer    64
Healthy        52
Name: count, dtype: int64


## 2. Data partitioning (Train / Validation / Test)

In [7]:

# 60/20/20 split: First split train vs temp; then temp -> val/test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=random_state
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (69, 9) Val: (23, 9) Test: (24, 9)


## 3. Feature inspection

In [8]:

# Distributions (train only)
fig = make_subplots(rows=2, cols=4, subplot_titles=list(X_train.columns), horizontal_spacing=0.08, vertical_spacing=0.15)
r, c = 1, 1
for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Histogram(x=X_train[col], nbinsx=20, name=col, showlegend=False), row=r, col=c)
    c += 1
    if c == 5:
        r += 1
        c = 1
fig.update_layout(height=600, width=1000, title_text="Train feature distributions")
fig.show()

# Correlation (Spearman) — train only
corr = X_train.corr(method="spearman")
fig = px.imshow(corr, text_auto=False, aspect="auto", color_continuous_scale="RdBu", origin="lower")
fig.update_layout(title="Spearman correlation (train only)")
fig.show()


Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

### 3.1 Feature ranking (Kruskal–Wallis & ROC-AUC on train)

In [ ]:

features = list(X_train.columns)
rank_rows = []
for f in features:
    # Kruskal on train
    h = kruskal(X_train.loc[y_train==1, f], X_train.loc[y_train==2, f])[0]
    # ROC-AUC on train (treat feature as score)
    auc = roc_auc_score(y_train, X_train[f])
    rank_rows.append({"Feature": f, "Kruskal H": h, "ROC-AUC": auc})

ranking = pd.DataFrame(rank_rows).sort_values(by="Kruskal H", ascending=False).reset_index(drop=True)
display(ranking)


,Feature,Kruskal H,ROC-AUC
0,Glucose,20.707548,0.820034
1,HOMA,5.308901,0.662139
2,Resistin,5.198302,0.660441
3,Insulin,3.059721,0.623090
4,Age,0.863639,0.434635
5,Adiponectin,0.139850,0.473684
6,MCP.1,0.083824,0.520374
7,BMI,0.067270,0.481749
8,Leptin,0.064177,0.482173


## 4. Standardization (fit on train only)

In [ ]:

scaler = StandardScaler().fit(X_train)
X_train_s = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_val_s   = pd.DataFrame(scaler.transform(X_val),   columns=X_val.columns,   index=X_val.index)
X_test_s  = pd.DataFrame(scaler.transform(X_test),  columns=X_test.columns,  index=X_test.index)

display(X_train_s.head())


,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1
105,0.985013,1.796462,1.415239,-0.499889,-0.280224,0.657185,-0.947464,-0.594132,0.770072
27,0.543676,0.718473,-0.431201,-0.108952,-0.221532,0.869254,0.033808,0.668101,1.660957
49,1.741590,-0.204125,-0.179414,-0.627060,-0.529939,-0.904772,-0.304982,-0.385835,-0.813956
52,-0.780335,-1.212292,0.072374,0.390091,0.222214,-0.914194,1.529756,0.641476,0.097494
50,1.174157,-0.108944,0.408090,1.728852,1.345461,-0.256463,-0.723803,-0.471534,-1.343475


## 5. PCA (train fit)

In [ ]:

pca = PCA().fit(X_train_s)
expl_var = pca.explained_variance_

fig = px.line(x=np.arange(1, len(expl_var)+1), y=expl_var,
              labels={"x":"PC", "y":"Explained variance"},
              title="Scree Plot (train)")
fig.add_hline(y=1, line_width=3, line_dash="dash", line_color="red")
fig.update_traces(mode="markers+lines")
fig.show()

kaiser = expl_var[expl_var > 1].sum() / expl_var.sum() * 100 if expl_var.sum() > 0 else 0.0
print(f"Variance retained by Kaiser: {kaiser:.2f}%")

# Project to 2D for visualization
pca2 = PCA(n_components=2).fit(X_train_s)
PC_train = pca2.transform(X_train_s)
PC_val   = pca2.transform(X_val_s)
PC_test  = pca2.transform(X_test_s)

fig = px.scatter(x=PC_train[:,0], y=PC_train[:,1],
                 color=y_train.replace({1:'Healthy', 2:'With Cancer'}),
                 labels={"x":"PC1","y":"PC2","color":"Class"},
                 title="PCA: PC1 vs PC2 (train)")
fig.update_traces(marker_size=9)
fig.show()


Variance retained by Kaiser: 77.50%


## 6. LDA (train fit) — 1D projection

In [ ]:

lda_all = LinearDiscriminantAnalysis().fit(X_train_s, y_train)
LD_train = lda_all.transform(X_train_s).ravel()
LD_val   = lda_all.transform(X_val_s).ravel()
LD_test  = lda_all.transform(X_test_s).ravel()

fig = px.scatter(x=LD_train, y=np.zeros_like(LD_train),
                 color=y_train.replace({1:'Healthy',2:'With Cancer'}),
                 labels={"x":"LDA1","y":""},
                 title="LDA1 — Train (1D projection)")
fig.update_yaxes(visible=False)
fig.update_traces(marker_size=9)
fig.show()


## 7. Classifiers — MDC (Euclidean & Mahalanobis) + Fisher LDA

In [ ]:

def metrics_dict(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    TP = np.sum((y_true==1)&(y_pred==1))
    TN = np.sum((y_true==2)&(y_pred==2))
    FP = np.sum((y_true==2)&(y_pred==1))
    FN = np.sum((y_true==1)&(y_pred==2))
    SS = TP/(TP+FN) if (TP+FN)>0 else 0.0
    SP = TN/(TN+FP) if (TN+FP)>0 else 0.0
    PR = TP/(TP+FP) if (TP+FP)>0 else 0.0
    F1 = 2*PR*SS/(PR+SS) if (PR+SS)>0 else 0.0
    AC = (TP+TN)/len(y_true)
    return {"Accuracy":AC, "Sensitivity":SS, "Specificity":SP, "Precision":PR, "F1":F1}

def mdc_euclid_predict(X, mu1, mu2):
    g1 = X @ mu1 - 0.5*(mu1.T@mu1)
    g2 = X @ mu2 - 0.5*(mu2.T@mu2)
    return np.where(g1>=g2, 1, 2)

def mdc_mahal_predict(X, mu1, mu2, Cinv):
    g1 = X @ (Cinv @ mu1) - 0.5*(mu1.T@Cinv@mu1)
    g2 = X @ (Cinv @ mu2) - 0.5*(mu2.T@Cinv@mu2)
    return np.where(g1>=g2, 1, 2)


### 7.1 Model selection on Validation (feature combos for MDC)

In [ ]:

features_all = list(X_train_s.columns)

def evaluate_mdc_combo(feats):
    # train stats
    Xh = X_train_s.loc[y_train==1, feats].to_numpy()
    Xc = X_train_s.loc[y_train==2, feats].to_numpy()
    mu1 = Xh.mean(axis=0, keepdims=True).T
    mu2 = Xc.mean(axis=0, keepdims=True).T
    C   = np.cov(np.vstack([Xh,Xc]).T)
    Cinv= np.linalg.pinv(C)  # pinv for stability

    # val metrics
    Xv = X_val_s[feats].to_numpy()
    yv = y_val.to_numpy()

    y_pred_e = mdc_euclid_predict(Xv, mu1, mu2)
    y_pred_m = mdc_mahal_predict(Xv, mu1, mu2, Cinv)

    m_e = metrics_dict(yv, y_pred_e)
    m_m = metrics_dict(yv, y_pred_m)

    return m_e, m_m

results_rows = []
# Evaluate 2-feature and 3-feature combos
for k in [2,3]:
    for feats in combinations(features_all, k):
        m_e, m_m = evaluate_mdc_combo(list(feats))
        results_rows.append({
            "Features": list(feats),
            "k": k,
            "Euclid_Acc": m_e["Accuracy"],
            "Euclid_F1":  m_e["F1"],
            "Mahalo_Acc": m_m["Accuracy"],
            "Mahalo_F1":  m_m["F1"],
        })

mdc_val_table = pd.DataFrame(results_rows).sort_values(by=["Mahalo_Acc","Euclid_Acc"], ascending=False)
display(mdc_val_table.head(10))


,Features,k,Euclid_Acc,Euclid_F1,Mahalo_Acc,Mahalo_F1
34,"[Adiponectin, MCP.1]",2,11.826087,0.411899,11.826087,0.411899
40,"[Age, BMI, Adiponectin]",3,11.434783,0.474308,11.826087,0.411899
11,"[BMI, Leptin]",2,11.304348,0.491493,11.695652,0.434783
5,"[Age, Adiponectin]",2,11.695652,0.434783,11.565217,0.455487
32,"[Leptin, MCP.1]",2,11.695652,0.434783,11.565217,0.455487
117,"[Leptin, Adiponectin, MCP.1]",3,11.695652,0.434783,11.565217,0.455487
7,"[Age, MCP.1]",2,11.565217,0.455487,11.565217,0.455487
30,"[Leptin, Adiponectin]",2,11.565217,0.455487,11.565217,0.455487
61,"[Age, Adiponectin, Resistin]",3,11.565217,0.455487,11.565217,0.455487
62,"[Age, Adiponectin, MCP.1]",3,11.565217,0.455487,11.565217,0.455487


### 7.2 Final evaluation on Test (using best Validation combo)

In [ ]:

# Pick best row by Mahalanobis Accuracy first, then Euclidean
best_row = mdc_val_table.iloc[0]
best_feats = best_row["Features"]
print("Best feature combo (by Val, preferring Mahalanobis):", best_feats)

# Train stats on train
Xh = X_train_s.loc[y_train==1, best_feats].to_numpy()
Xc = X_train_s.loc[y_train==2, best_feats].to_numpy()
mu1 = Xh.mean(axis=0, keepdims=True).T
mu2 = Xc.mean(axis=0, keepdims=True).T
C   = np.cov(np.vstack([Xh,Xc]).T)
Cinv= np.linalg.pinv(C)

# Test predictions
Xt = X_test_s[best_feats].to_numpy()
yt = y_test.to_numpy()
y_pred_e_test = mdc_euclid_predict(Xt, mu1, mu2)
y_pred_m_test = mdc_mahal_predict(Xt, mu1, mu2, Cinv)

print("\n== MDC (Euclidean) — Test ==")
print(metrics_dict(yt, y_pred_e_test))
print("\n== MDC (Mahalanobis) — Test ==")
print(metrics_dict(yt, y_pred_m_test))


Best feature combo (by Val, preferring Mahalanobis): ['Adiponectin', 'MCP.1']

== MDC (Euclidean) — Test ==
{'Accuracy': 12.5, 'Sensitivity': 0.25, 'Specificity': 0.75, 'Precision': 0.4583333333333333, 'F1': 0.3235294117647059}

== MDC (Mahalanobis) — Test ==
{'Accuracy': 12.333333333333334, 'Sensitivity': 0.3333333333333333, 'Specificity': 0.6666666666666666, 'Precision': 0.4583333333333333, 'F1': 0.38596491228070173}


### 7.3 Fisher LDA — Validation and Test

In [ ]:

# Evaluate LDA with: (a) best_feats from MDC search; (b) all features
def eval_lda(feats):
    clf = LinearDiscriminantAnalysis()
    clf.fit(X_train_s[feats], y_train)
    yv = clf.predict(X_val_s[feats])
    mv = metrics_dict(y_val, yv)
    yt = clf.predict(X_test_s[feats])
    mt = metrics_dict(y_test, yt)
    return mv, mt

mv_best, mt_best = eval_lda(best_feats)
mv_all,  mt_all  = eval_lda(features_all)

print("LDA (best_feats) — Val:", mv_best)
print("LDA (best_feats) — Test:", mt_best)
print("\nLDA (all features) — Val:", mv_all)
print("LDA (all features) — Test:", mt_all)


LDA (best_feats) — Val: {'Accuracy': 0.6521739130434783, 'Sensitivity': 0.3, 'Specificity': 0.9230769230769231, 'Precision': 0.75, 'F1': 0.4285714285714285}
LDA (best_feats) — Test: {'Accuracy': 0.4166666666666667, 'Sensitivity': 0.0, 'Specificity': 0.7692307692307693, 'Precision': 0.0, 'F1': 0.0}

LDA (all features) — Val: {'Accuracy': 0.6956521739130435, 'Sensitivity': 1.0, 'Specificity': 0.46153846153846156, 'Precision': 0.5882352941176471, 'F1': 0.7407407407407407}
LDA (all features) — Test: {'Accuracy': 0.6666666666666666, 'Sensitivity': 0.8181818181818182, 'Specificity': 0.5384615384615384, 'Precision': 0.6, 'F1': 0.6923076923076923}


## 8. Visualizations — MDC on top 2 features

In [ ]:

# If best_feats has 2+, plot first two
if len(best_feats) >= 2:
    f1, f2 = best_feats[:2]
    Xh2 = X_train_s.loc[y_train==1, [f1,f2]].to_numpy()
    Xc2 = X_train_s.loc[y_train==2, [f1,f2]].to_numpy()
    mu1_2 = Xh2.mean(axis=0, keepdims=True).T
    mu2_2 = Xc2.mean(axis=0, keepdims=True).T
    C2 = np.cov(np.vstack([Xh2,Xc2]).T)
    C2i = np.linalg.pinv(C2)

    # grid
    Xplot = X_train_s[[f1,f2]].to_numpy()
    x_min, x_max = Xplot[:,0].min()-0.5, Xplot[:,0].max()+0.5
    y_min, y_max = Xplot[:,1].min()-0.5, Xplot[:,1].max()+0.5

    # scatter points
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_train_s.loc[y_train==1, f1], y=X_train_s.loc[y_train==1, f2],
                             mode="markers", name="Healthy", marker=dict(size=8)))
    fig.add_trace(go.Scatter(x=X_train_s.loc[y_train==2, f1], y=X_train_s.loc[y_train==2, f2],
                             mode="markers", name="With Cancer", marker=dict(size=8)))

    # Euclidean boundary: (w^T x + b = 0). For two means, line between means perpendicular bisector.
    W = (mu1_2 - mu2_2)
    b = -0.5 * (mu1_2 - mu2_2).T @ (mu1_2 + mu2_2)
    xx = np.linspace(x_min, x_max, 200)
    yy = -(W[0,0]/W[1,0])*xx - b[0,0]/W[1,0]

    fig.add_trace(go.Scatter(x=xx, y=yy, mode="lines",
                             line=dict(color="gray", dash="dash"), name="Euclid boundary"))

    fig.update_layout(title=f"MDC (Euclidean) — Decision boundary on [{f1}, {f2}]",
                      xaxis_title=f1, yaxis_title=f2,
                      width=900, height=700)
    fig.show()
else:
    print("Best combo has <2 features — skipping 2D boundary plot.")



## 9. Summary & Notes

- **No leakage**: all selection/fit steps use only the training set; validation/test are transformed with the same fitted objects.
- **Feature selection**: use `ranking` (Kruskal/ROC-AUC) to justify chosen sets; avoid pairs with very high correlation.
- **Dimensionality reduction**: PCA (variance/scree + plots) and LDA (1D projection) are included for analysis and potential pipelines.
- **Classifiers**: both **MDC Euclidean** and **MDC Mahalanobis** are implemented and compared, with **Fisher LDA** as well.
- **Model choice**: select on validation (`mdc_val_table`, `LDA` val metrics), then report **final test** only once with the chosen model.

**Ideas to try next**
- Balance classes via class-weight (for LDA) or stratified resampling.
- Nested evaluation with repeated CV on **train** to reduce variance of selection (then still keep a final untouched test).
- Try MDC only com 2–3 features que maximizem *Specificity* caso o objetivo seja reduzir falsos positivos (ou outra métrica clínica relevante).
